In [0]:
#!pip install hyperopt

In [23]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from skimage import color, exposure

from sklearn.metrics import accuracy_score


from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
tests = pd.read_pickle('data/test.p')

In [10]:
X_train, y_train = train['features'], train['labels']
X_test, y_test = tests['features'], tests['labels']
X_train.shape, y_train.shape

((34799, 32, 32, 3), (34799,))

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])
  #logir sciezka do katalogu z logami
  logdir= os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d=%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size',128), #podzial na paczki
      epochs = params_fit.get('epochs',5), #liczba iteracji dla paczek
      verbose = params_fit.get('verbose',1), #widocznosc stanu
      validation_data=params_fit.get('validation_data',(X_train,y_train)), # wizualizacja
      callbacks=[tensorboard_callback]
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scorong= accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob,axis=1) # axis=1  needed to find the max prob for each sign not for whole set

 
  return scorong(y_test_norm,y_pred)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  return Sequential ([
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu', input_shape=input_shape),
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu',padding='same' ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),
 
    Dense(1024, activation = 'relu'),
    Dropout(0.3),

    Dense(1024, activation = 'relu'),
    Dropout(0.3),

    Dense(num_classes, activation = 'softmax'),
  ])

In [20]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train,y_train)
predict(model_trained, X_test, y_test )

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.6016 - accuracy: 0.2930 - val_loss: 0.9511 - val_accuracy: 0.6930
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.8082 - accuracy: 0.7417 - val_loss: 0.2492 - val_accuracy: 0.9297
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.4033 - accuracy: 0.8718 - val_loss: 0.1107 - val_accuracy: 0.9638
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2646 - accuracy: 0.9184 - val_loss: 0.0504 - val_accuracy: 0.9864
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1992 - accuracy: 0.9388 - val_loss: 0.0435 - val_accuracy: 0.9861


0.9634920634920635

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape,num_classes)
  model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs=7,
      #epochs=params.get('epochs',5),
      verbose=0
     
    )
  accuracy= model.evaluate(X_test,y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))
  #=0.8 vs -1.0, min(-0.8,-1) = -1, min if no "-" can cause it will look for values closes to 0 which is the worse result
  return {'loss' : -accuracy, 'status': STATUS_OK, 'model': model}

In [34]:
space = {
    'batch_size': hp.quniform('batch_size',50,200,20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 1.9876 - accuracy: 0.2500

 10/138 [=>............................]
 - ETA: 0s - loss: 0.2348 - accuracy: 0.9187

 17/138 [==>...........................]
 - ETA: 0s - loss: 0.1838 - accuracy: 0.9357

 24/138 [====>.........................]
 - ETA: 0s - loss: 0.2471 - accuracy: 0.9206

 32/138 [=====>........................]
 - ETA: 0s - loss: 0.1906 - accuracy: 0.9385

 35/138 [======>.......................]
 - ETA: 0s - loss: 0.1752 - accuracy: 0.9438


In [0]:
def get_model(params):
  return Sequential ([
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu', input_shape=input_shape),
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu',padding='same' ),
    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ),
    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters= 128, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 128, kernel_size=(3,3), activation = 'relu' ),
    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
 
    Dense(1024, activation = 'relu'),
    #Dropout(0.3),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation = 'relu'),
    #Dropout(0.3),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation = 'softmax'),
  ])

In [0]:
def func_obj2(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs=5,
      #epochs=params.get('epochs',5),
      verbose=0
     
    )
  accuracy= model.evaluate(X_test,y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))
  #=0.8 vs -1.0, min(-0.8,-1) = -1, min if no "-" can cause it will look for values closes to 0 which is the worse result
  return {'loss' : -accuracy, 'status': STATUS_OK, 'model': model}

In [47]:
space = {
    'batch_size': hp.quniform('batch_size',50,200,20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0,1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0,1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0,1),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0,1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0,1),
}

best = fmin(
    func_obj2,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.2491 - accuracy: 0.9062

 11/138 [=>............................]
 - ETA: 0s - loss: 0.1576 - accuracy: 0.9489

 19/138 [===>..........................]
 - ETA: 0s - loss: 0.3290 - accuracy: 0.9062

 26/138 [====>.........................]
 - ETA: 0s - loss: 0.3543 - accuracy: 0.9002

 34/138 [======>.......................]
 - ETA: 0s - loss: 0.2803 - accuracy: 0.9200

 42/138 [========>.....................]
 - ETA: 0s - loss: 0.2441 - accuracy: 0.9308


In [0]:
#https://arxiv.org/abs/1711.04623 stochastic gradient descent jastrzebski
#https://arxiv.org/pdf/1711.04623.pdf learning rate and batch size dependency
#Input:
#best = fmin(
#    func_obj2,
#    space,
#   tpe.suggest,
#   1,
#    Trials()
#)

#Output:
#{'batch_size': 140.0, 'dropout_cnn_block_one': 0.9224283278581105, 'dropout_cnn_block_three': 0.578623185950145, 'dropout_cnn_block_two': 0.23061228581111026, 'dropout_dense_block_one': 0.3653059452047387, 'dropout_dense_block_two': 0.9313136086125292}
#accuracy=3.5520973205566406
#100%|██████████| 1/1 [00:13<00:00, 13.01s/it, best loss: -3.5520973205566406]

In [0]:
#https://arxiv.org/abs/1711.04623 stochastic gradient descent jastrzebski
#https://arxiv.org/pdf/1711.04623.pdf learning rate and batch size dependency
#Input:
#best = fmin(
#    func_obj2,
#    space,
#   tpe.suggest,
#   30,
#    Trials()
#)

#Output:
#{'batch_size': 100.0, 'dropout_cnn_block_one': 0.6720590247070197, 'dropout_cnn_block_three': 0.5644965608861164, 'dropout_cnn_block_two': 0.6483027221354546, 'dropout_dense_block_one': 0.49766755889265124, 'dropout_dense_block_two': 0.8204465794422495}
#accuracy=2.2205862998962402
#100%|██████████| 30/30 [06:46<00:00, 13.55s/it, best loss: -3.5548095703125]

In [0]:
def get_model2(params):
  return Sequential ([
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu', input_shape=input_shape),
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu',padding='same' ),
    Conv2D(filters= 32, kernel_size=(3,3), activation = 'relu',padding='same' ),
    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu' ),
    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters= 128, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 128, kernel_size=(3,3), activation = 'relu' ,padding='same'),
    Conv2D(filters= 128, kernel_size=(3,3), activation = 'relu' ),

    MaxPool2D(),
    #Dropout(0.3),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
 
    Dense(1024, activation = 'relu'),
    #Dropout(0.3),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation = 'relu'),
    #Dropout(0.3),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation = 'softmax'),
  ])

In [0]:
def func_obj3(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = 125, #int(params.get('batch_size',128)),
      epochs=5,
      #epochs=params.get('epochs',5),
      verbose=0
     
    )
  score= model.evaluate(X_test,y_test,verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  #=0.8 vs -1.0, min(-0.8,-1) = -1, min if no "-" can cause it will look for values closes to 0 which is the worse result
  return {'loss' : -accuracy, 'status': STATUS_OK, 'model': model}

In [54]:
space = {
    'batch_size': hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.7),
}

best = fmin(
    func_obj3,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3206418421902899, 'dropout_cnn_block_three': 0.4848740079075806, 'dropout_cnn_block_two': 0.37155980333699035, 'dropout_dense_block_one': 0.5117355622944653, 'dropout_dense_block_two': 0.636266284700336}
accuracy=0.9029478430747986
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4728475147694274, 'dropout_cnn_block_three': 0.36750623432838964, 'dropout_cnn_block_two': 0.3852077819393711, 'dropout_dense_block_one': 0.4820026868313051, 'dropout_dense_block_two': 0.38801333772194924}
accuracy=0.9442176818847656
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4689341789921098, 'dropout_cnn_block_three': 0.3570377655199142, 'dropout_cnn_block_two': 0.46570218713515044, 'dropout_dense_block_one': 0.3527018812512991, 'dropout_dense_block_two': 0.38328332625578093}
accuracy=0.8891156315803528
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4039255483421567, 'dropout_cnn_block_three': 0.42434554327104573, 'dropout_cnn_block_two': 0.42377569323